In [1]:
from pyspark.sql import SparkSession

In [5]:
spark_create = SparkSession.builder.master("local").appName("Word Count")\
.config("spark.some.config.option", "some-value")\
.getOrCreate()

In [4]:
spark

In [6]:
spark_create

In [8]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [19]:
# in Python

df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)

step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect() # 2500000000000

[Row(sum(id)=2500000000000)]

In [13]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) HashAggregate(keys=[], functions=[sum(id#29L)])
   +- ShuffleQueryStage 4
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=555]
         +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#29L)])
            +- *(6) Project [id#29L]
               +- *(6) SortMergeJoin [id#29L], [id#23L], Inner
                  :- *(4) Sort [id#29L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2
                  :        +- Exchange hashpartitioning(id#29L, 200), ENSURE_REQUIREMENTS, [plan_id=426]
                  :           +- *(3) Project [(id#21L * 5) AS id#29L]
                  :              +- ShuffleQueryStage 0
                  :                 +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=359]
                  :                    +- *(1) Range (2, 10000000, step=2, splits=8)
            

In [11]:
spark